In [70]:
import os, torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split

'''
All Dataset class
'''
# 타이타닉 "전체 데이터셋" 사용 위한 클래스
class TitanicDataset(Dataset):
    def __init__(self, X, y): # Initial Setting
        self.X = torch.FloatTensor(X) # 입력 데이터
        # 입력 데이터를 부동 소수점 형태로 변환하는 이유는
        # 입력 데이터가 모델의 가중치와 곱셈 연산을 수행해
        # 소수점 정밀도가 필요하기 때문이다.
        
        self.y = torch.LongTensor(y) # 출력 데이터
        # 출력 데이터는 입력 데이터와 달리
        # 별다른 계산을 수행하지 않고 분류만 하므로
        # 정밀도가 필요하지 않고 정수를 사용하는 것이 효과적이다.
    
    # 데이터셋 sample 수 반환
    def __len__(self): return len(self.X)
    
    # 주어진 idx에 해당하는 샘플을 가져옴
    def __getitem__(self, idx):
        feature, target = self.X[idx], self.y[idx]
        return {'input': feature, 'target': target} #딕셔너리로 반환

    # dataset String 표현
    def __str__(self):
        # 데이터 크기, 입력 형태, 타겟 형태 출력
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )
        return str

In [71]:
'''
Test Dataset Class
테스트용이므로 정답 레이블(target)을 포함하지 않음
'''
# 타이타닉 "테스트 데이터셋" 사용 위한 클래스 
class TitanicTestDataset(Dataset):
    def __init__(self, X): # Initial Setting
        self.X = torch.FloatTensor(X) # 입력 데이터
    
    def __len__(self): return len(self.X) # 데이터셋 sample 수 반환

    # 주어진 idx에 해당하는 샘플을 가져옴
    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature} #딕셔너리로 반환

    # dataset String 표현
    def __str__(self):
        # 데이터 크기, 입력 형태, 타겟 형태 출력
        str = "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )
        return str

In [72]:
def get_preprocessed_dataset():
    # Load the paths of datasets
    # CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__)) # 주피터 노트북에서는 __file__ 허용되지 않음
    CURRENT_FILE_PATH = os.getcwd()  # 현재 작업 디렉토리 경로 가져오기
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    # Load datasets
    train_df = pd.read_csv(train_data_path) # train dataset
    test_df = pd.read_csv(test_data_path) # test dataset
    
    # train + test (No sort)
    all_df = pd.concat([train_df, test_df], sort=False)
    
    # Preprocessing
    all_df = get_preprocessed_dataset_1(all_df) # 1회
    all_df = get_preprocessed_dataset_2(all_df) # 2회
    all_df = get_preprocessed_dataset_3(all_df) # 3회
    all_df = get_preprocessed_dataset_4(all_df) # 4회
    all_df = get_preprocessed_dataset_5(all_df) # 5회
    all_df = get_preprocessed_dataset_6(all_df) # 6회
    
    # 학습 및 테스트 데이터 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    #print(test_dataset)
    # 학습 데이터셋을 생성하고 훈련 및 검증 데이터셋으로 분할.
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # 테스트 데이터셋 생성
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset

In [73]:
'''
Preprocessing
'''
# 여러 단계의 데이터 전처리 함수 정의
def get_preprocessed_dataset_1(all_df):
    # Pclass별로 Fare 평균값을 사용하여 결측치를 채움
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    return all_df

def get_preprocessed_dataset_2(all_df):
    # Name 컬럼을 family_name, honorific, name으로 분리하여 추가합니다.
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)
    return all_df

def get_preprocessed_dataset_3(all_df):
    # honorific별로 Age 평균값을 사용하여 결측치를 채움
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)
    return all_df

def get_preprocessed_dataset_4(all_df):
    # 가족 수와 혼자 탑승 여부 컬럼 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)
    return all_df

def get_preprocessed_dataset_5(all_df):
    # honorific 컬럼의 값 개수 줄이기
    all_df.loc[
        ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
        ),
        "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)
    return all_df

def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수들을 LabelEncoder를 사용하여 수치형으로 변환
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])
    return all_df

In [74]:
from torch import nn, optim
'''
사용자 정의 신경망 모델
'''
class ActivationFunc(nn.Module) :
    def __init__(self):
        super().__init__()
        
        # activate function 정의
        self.activate_dir_ = {'ReLU' : nn.ReLU,
                         'Leaky ReLU' : nn.LeakyReLU,
                        'Sigmoid' : nn.Sigmoid,
                        'Tanh' : nn.Tanh,
                        'SoftPlus' : nn.Softplus,
                        'ELU' : nn.ELU,
                        'GELU' : nn.GELU,
                        'SiLU' : nn.SiLU,
                        'Mish' : nn.Mish}
        
        self.activate_list_ = list(self.activate_dir_.values())
    
    def get_activation_func(self, name = None, idx = None) :
        if (name is not None) : return self.activate_dir_[name]()
        elif (idx is not None) : return self.activate_list_[idx]()
        else : return nn.ReLU() #Default

# 사용자 정의 신경망 모델 클래스 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):  # 초기 설정
        super().__init__()  # 부모 클래스 초기화
        actFunc_ = ActivationFunc()
        # 모델 구조 정의 (은닉층 2개 포함)
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),  # 입력층 -> 첫 번째 은닉층
            # nn.ReLU(),  # 활성화 함수: ReLU
            actFunc_.get_activation_func(name = 'ELU'),
            nn.Linear(30, 30),  # 첫 번째 은닉층 -> 두 번째 은닉층
            # nn.ReLU(),  # 활성화 함수: ReLU
            actFunc_.get_activation_func(name = 'ELU'),
            nn.Linear(30, n_output),  # 두 번째 은닉층 -> 출력층
        )
    
    def forward(self, x):  # 순전파 함수
        return self.model(x)  # 정의된 모델을 통해 순전파 수행
    
# 모델 및 옵티마이저를 가져오는 함수
def get_model_and_optimizer(input_size, output_size):
    model = MyModel(input_size, output_size)  # 사용자 정의 모델 인스턴스 생성
    optimizer = optim.SGD(model.parameters(), lr=0.01)  # 옵티마이저: 확률적 경사 하강법 (학습률 0.01)
    return model, optimizer


In [75]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from datetime import datetime
import wandb

# 학습 루프 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    #n_epochs = wandb.config.epochs  # 에포크 수를 wandb 설정에서 가져옴
    n_epochs = 700
    loss_fn = nn.CrossEntropyLoss()  # 손실 함수로 교차 엔트로피 손실 사용
    next_print_epoch = 100  # 다음 출력할 에포크 설정

    for epoch in range(1, n_epochs + 1):
        model.train()  # 모델을 학습 모드로 설정
        loss_train = 0.0  # 학습 손실 초기화
        correct_train = 0  # 정확히 예측한 학습 샘플 수 초기화
        total_train = 0  # 전체 학습 샘플 수 초기화
        for batch in train_data_loader:  # 학습 데이터 로더에서 배치 반복
            inputs, targets = batch['input'], batch['target']  # 입력과 타겟 데이터 가져오기
            outputs = model(inputs)  # 모델을 사용해 예측값 계산
            loss = loss_fn(outputs, targets)  # 손실 함수 계산
            loss_train += loss.item()  # 손실 누적

            optimizer.zero_grad()  # 경사 초기화
            loss.backward()  # 역전파를 통해 경사 계산
            optimizer.step()  # 경사를 사용해 가중치 업데이트

            _, predicted = torch.max(outputs, 1)  # 가장 높은 값을 가지는 클래스 예측
            correct_train += (predicted == targets).sum().item()  # 정확히 예측한 개수 누적
            total_train += targets.size(0)  # 전체 샘플 수 누적

        # 검증 손실 및 정확도 계산
        loss_validation = 0.0  # 검증 손실 초기화
        correct_val = 0  # 정확히 예측한 검증 샘플 수 초기화
        total_val = 0  # 전체 검증 샘플 수 초기화
        model.eval()  # 모델을 평가 모드로 설정
        with torch.no_grad():  # 경사 계산 비활성화 (평가 시 필요 없음)
            for batch in validation_data_loader:  # 검증 데이터 로더에서 배치 반복
                inputs, targets = batch['input'], batch['target']  # 입력과 타겟 데이터 가져오기
                outputs = model(inputs)  # 모델을 사용해 예측값 계산
                loss = loss_fn(outputs, targets)  # 손실 함수 계산
                loss_validation += loss.item()  # 손실 누적

                _, predicted = torch.max(outputs, 1)  # 가장 높은 값을 가지는 클래스 예측
                correct_val += (predicted == targets).sum().item()  # 정확히 예측한 개수 누적
                total_val += targets.size(0)  # 전체 샘플 수 누적

        train_accuracy = correct_train / total_train  # 학습 정확도 계산
        val_accuracy = correct_val / total_val  # 검증 정확도 계산

        # Wandb에 로그 기록
        wandb.log({
            "Epoch": epoch,  # 현재 에포크
            "Training loss": loss_train / len(train_data_loader),  # 에포크당 평균 학습 손실
            "Validation loss": loss_validation / len(validation_data_loader)  # 에포크당 평균 검증 손실
        })

        # 특정 에포크마다 학습 및 검증 손실 출력
        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / len(train_data_loader):.4f}, "
                f"Validation loss {loss_validation / len(validation_data_loader):.4f}"
            )
            # print(f"Testing at epoch {epoch}")
            # test_model(model, test_data_loader)
            next_print_epoch += 100


In [76]:
'''
Test
'''
# 테스트 함수
def test_model(model, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    print("[TEST]")
    correct_test = 0
    total_test = 0
    results = []
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch['input']  # 입력 데이터 가져오기
            outputs = model(inputs)  # 모델 출력 계산
            predictions = torch.argmax(outputs, dim=1)  # 예측값 계산
            # 예측 결과 출력 및 저장 (인덱스는 892부터 시작)
            for idx, prediction in enumerate(predictions, start=892):
                print(idx, prediction.item())
                results.append({'PassengerId': idx, 'Survived': prediction.item()})
            # 테스트 데이터에 대해 정확도 평가 (타겟 값은 없으므로 실제 예측 정확도 계산 불가)
            correct_test += (predictions == 1).sum().item()  # 임시로 1로 설정된 기준에 따른 평가
            total_test += predictions.size(0)  # 전체 샘플 수 누적
    test_accuracy = correct_test / total_test  # 테스트 정확도 계산
    print(f"Test accuracy: {test_accuracy:.4f}")

    # 현재 작업 디렉토리를 출력하여 확인
    current_directory = os.getcwd()
    print(f"Current working directory: {current_directory}")

    # 절대 경로로 파일을 저장
    results_df = pd.DataFrame(results)
    file_path = os.path.join(current_directory, 'submission.csv')
    results_df.to_csv(file_path, index=False)
    print(f"Test results saved to {file_path}")


In [77]:
'''
Main Function
'''
if __name__ == "__main__":
    # Wandb 설정 초기화
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')  # 현재 시간 문자열로 변환

    config = {
        'epochs': 1000,  # 총 학습 에포크 수
        'batch_size': 16,  # 배치 크기
        'learning_rate': 1e-3,  # 학습률
    }

    wandb.init(
        mode="online",  # 온라인 모드로 설정 (wandb 로그 기록 활성화)
        project="titanic_model_training_ELU",  # 프로젝트 이름
        notes="Titanic dataset training with wandb",  # 실험에 대한 설명
        tags=["titanic", "classification"],  # 태그 설정
        name=current_time_str,  # 실험 이름 (현재 시간)
        config=config  # 설정값 전달
    )

    # 데이터셋 로드 및 분할
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()  # 전처리된 데이터셋 가져오기
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)  # 학습 데이터 로더
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))  # 검증 데이터 로더
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))  # 테스트 데이터 로더
    
    # 모델 및 옵티마이저 정의
    input_size = train_dataset[0]['input'].shape[0]  # 입력 특성 수
    output_size = 2  # 출력 클래스 수 (생존 여부: 0 또는 1)
    model = MyModel(input_size, output_size)  # 모델 인스턴스 생성
    optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate)  # 옵티마이저 설정 (SGD)

    # 학습 루프 실행
    training_loop(model, optimizer, train_data_loader, validation_data_loader)

    # 테스트 데이터셋에 대해 모델 평가
    test_model(model, test_data_loader)
    
    wandb.finish()  # wandb 세션 종료
    

C:\Users\yscho\AppData\Local\Temp\ipykernel_17252\3076189289.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\yscho\AppData\Local\Temp\ipykernel_17252\3076189289.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

Epoch 100, Training loss 0.5519, Validation loss 0.5531
Epoch 200, Training loss 0.5155, Validation loss 0.5683
Epoch 300, Training loss 0.4863, Validation loss 0.5532
Epoch 400, Training loss 0.4488, Validation loss 0.4979
Epoch 500, Training loss 0.4301, Validation loss 0.5000
Epoch 600, Training loss 0.4251, Validation loss 0.4841
Epoch 700, Training loss 0.4140, Validation loss 0.5534
[TEST]
892 0
893 0
894 0
895 0
896 0
897 0
898 0
899 0
900 1
901 0
902 0
903 0
904 1
905 0
906 0
907 1
908 0
909 0
910 0
911 0
912 0
913 0
914 1
915 0
916 0
917 0
918 1
919 0
920 0
921 0
922 0
923 0
924 0
925 0
926 0
927 0
928 0
929 0
930 0
931 1
932 0
933 0
934 0
935 1
936 0
937 0
938 0
939 0
940 0
941 0
942 0
943 0
944 0
945 0
946 0
947 0
948 0
949 0
950 0
951 1
952 0
953 0
954 0
955 0
956 0
957 0
958 0
959 0
960 0
961 0
962 0
963 0
964 0
965 0
966 1
967 0
968 0
969 0
970 0
971 0
972 0
973 0
974 0
975 0
976 0
977 0
978 0
979 0
980 0
981 1
982 0
983 0
984 1
985 0
986 0
987 0
988 0
989 0
990 0
991 0
9

Epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
Training loss,███▇▇▇▇▇▆▆▆▆▅▅▄▄▄▄▄▃▃▄▃▂▂▃▂▁▂▃▂▂▂▂▂▂▂▁▁▁
Validation loss,▇██▇▆▆▆▅▇▅▅▅▄▆▆▄▃▄▅▃▆▅▅▃▂▃▄▄▂▃▂▁▄▄▂█▃▂▁▁
Epoch,700
Training loss,0.41398
Validation loss,0.55344


# 결과
![Activation Function Image](https://raw.githubusercontent.com/yuuun03/DL_choyunseo_3_2_/dacb1ee9748c98a77aa3ac3bc41b7bbfae10ef96/wandb_activationFunc.png)

본 그래프에서 각 시간-색깔이 의미하는 활성화 함수는 다음과 같다.
1. Mish : 2024-10-18_09-18-42 (파란색)
2. ReLU : 2024-10-18_18-23-14 (빨간색)
3. Leaky ReLU : 2024-10-18_18-27-22 (초록색)
4. Sigmoid : 2024-10-18_18-30-03 (보라색)
5. Tanh : 2024-10-18_18-33-15 (분홍색)
6. Softplus : 2024-10-18_18-35-34 (주황색)
7. ELU : 2024-10-18_18-37-39 (민트색)
8. GELU : 2024-10-18_18-39-22 (자주색)
9. SiLU : 2024-10-18_18-42-49 (노란색)

Validation Loss가 낮고 안정적일 수록 성능이 좋은 것이므로 성능이 좋은 순서대로 활성화함수를 다시 나열하면 다음과 같다. 
1. **<span style="color: #3EB489;">ELU : 2024-10-18_18-37-39 (민트색)</span>**
2. SiLU : 2024-10-18_18-42-49 (노란색)
3. ReLU : 2024-10-18_18-23-14 (빨간색)
4. GELU : 2024-10-18_18-39-22 (자주색)
5. Tanh : 2024-10-18_18-33-15 (분홍색)
6. Mish : 2024-10-18_09-18-42 (파란색)
7. Softplus : 2024-10-18_18-35-34 (주황색)
8. Leaky ReLU : 2024-10-18_18-27-22 (초록색)
9. **<span style="color: #8E44AD;">Sigmoid : 2024-10-18_18-30-03 (보라색)</span>**


# 고찰
## 제일 성능이 좋았던 함수 : ELU
음수 입력에 대해 선형적으로 비선형성을 적용하여 기울기 소실 문제를 완화하는 것이 ELU의 특징이다. 또한 학습 속도가 빠르기 때문에 성능이 높게 나올 수 있다.
## 제일 성능이 안 좋았던 함수 : Sigmoid
Sigmoid 함수는 출력이 (0, 1) 사이로 제한되어 있고, 큰 입력값에서 기울기가 매우 작아져 기울기 소실 문제가 발생하기 쉽다. 이 때문에 학습이 느리며 복잡한 비선형 문제에서 성능이 떨어진다.

# Epoch에 대한 고찰
### Epoch 700일 때 멈추는 것이 가장 낫다.
[ELU 사용.]
**가장 낮은 Validation loss는 Epoch 500**일 때이나 **Test Accuracy가 가장 높을 때는 Epoch 700**일 때이다. 두 에폭 간의 Validation loss의 차이가 그렇게 크지 않으므로 Test Accuracy의 높음 정도를 더 중요시 하는 것이 옳다고 판단하였다. 다만 에폭 700 이후에는 Validation loss도 Test Accuracy도 크게 개선되는 점이 없으므로 이 때 멈추는 것이 가장 좋다고 생각하였다.


# Kaggle 결과
![Kaggle결과_조윤서](https://github.com/yuuun03/DL_choyunseo_3_2_/raw/c0bd4852bd042178bee2ed00a6c5501c7ff6a51d/kaggle_leaderboard_2022136117%EC%A1%B0%EC%9C%A4%EC%84%9C.png)

# 숙제 후기
타이타닉 데이터셋은 스크립트 프로그래밍과 머신러닝 때도 사용해보았던 데이터셋입니다. 그 당시에는 그냥 pandas, seaborn을 사용한 가공 및 시각화를 하거나 직접 numpy로만 MLP를 이용하여서 분석해보았었는데 이번에 딥러닝 시간에 현업에서 주로 사용하는 딥러닝 모듈(또는 사이트)을 사용하여 데이터를 분석해보아서 좋았습니다. 그러면서 Kaggle, WandB라는 새로운 AI 사이트를 알게 되어 좋았습니다. 특히 WandB의 경우 사용해보니 너무 간편하게 시각화를 해주어 앞으로도 딥러닝을 이용한 개발을 하게 된다면 유용하게 사용할 것 같습니다.